### Loading Dataset

In [4]:
!pip install opendatasets
import opendatasets as od
import numpy as np
import pandas as pd
import sys
import time
import pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import re

class User:
    def __init__(self, id, age, sex, occupation, zip):
        self.id = int(id)
        self.age = int(age)
        self.sex = sex
        self.occupation = occupation
        self.zip = zip
        self.avg_r = 0.0

class Item:
    def __init__(self, id, title, release_date, video_release_date, imdb_url, \
    unknown, action, adventure, animation, childrens, comedy, crime, documentary, \
    drama, fantasy, film_noir, horror, musical, mystery ,romance, sci_fi, thriller, war, western):
        self.id = int(id)
        self.title = title
        self.release_date = release_date
        self.video_release_date = video_release_date
        self.imdb_url = imdb_url
        self.unknown = int(unknown)
        self.action = int(action)
        self.adventure = int(adventure)
        self.animation = int(animation)
        self.childrens = int(childrens)
        self.comedy = int(comedy)
        self.crime = int(crime)
        self.documentary = int(documentary)
        self.drama = int(drama)
        self.fantasy = int(fantasy)
        self.film_noir = int(film_noir)
        self.horror = int(horror)
        self.musical = int(musical)
        self.mystery = int(mystery)
        self.romance = int(romance)
        self.sci_fi = int(sci_fi)
        self.thriller = int(thriller)
        self.war = int(war)
        self.western = int(western)

class Rating:
    def __init__(self, user_id, item_id, rating, time):
        self.user_id = int(user_id)
        self.item_id = int(item_id)
        self.rating = int(rating)
        self.time = time

In [6]:
username:"neginmohtaram"
key:"a8c673e77557ed6d7bbb29b838c3816c"
od.download(
    'https://www.kaggle.com/datasets/prajitdatta/movielens-100k-dataset/ml-100k/u.data')

Skipping, found downloaded files in "./movielens-100k-dataset" (use force=True to force download)


In [7]:
def load_users(ratings):
    users = pd.unique(ratings['user_id'])
    return users

In [8]:
def load_ratings(file):
    R = []
    ratings = pd.read_csv(file, sep="\t", header=None, encoding='latin-1', names= ['user_id', 'movie_id', 'rating', 'unix_timestamp'])

    return  ratings 

In [9]:
def load_items(file):
    movies = pd.unique(ratings['movie_id'])
    return  movies

In [10]:
ratings = load_ratings("/content/movielens-100k-dataset/ml-100k/ua.base")
ratings_test = load_ratings("/content/movielens-100k-dataset/ml-100k/ua.test")
users = load_users(ratings)
items = load_items(ratings)
n_users = len(users)
n_items = len(items)
sampleUser = np.random.randint(n_users)
sampleMovie = np.random.randint(n_items)
train = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0).values
test = ratings_test.pivot(index='user_id', columns='movie_id', values='rating').fillna(0).values

In [71]:
a = []
for i in range(test.shape[1]):
        for j in range(test.shape[0]):
            if test[j][i] != 0:
                a.append((j, i))
test_data = np.array(a)

### Question 3

In [50]:
from tqdm.notebook import tqdm
def matrix_factorization(R, P, Q, K, alpha=0.005, lambda_=0.0005, steps=50):

    E = R - np.dot(P, Q)

    for step in tqdm(range(steps)):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    for k in range(K):
                        P[i][k] +=  alpha * (2 * E[i][j] * Q[k][j] - lambda_ * P[i][k])
                        Q[k][j] += alpha * (2 * E[i][j] * P[i][k] - lambda_ * Q[k][j])


        e = 0
        R_hat = np.dot(P,Q)
        R_hat[R == 0] = 0
        E = R - R_hat
        e = np.sum(np.power(E,2))  + np.sum(np.power(Q,2)) + (lambda_/2) * np.sum(np.power(P,2))
        if (step) % 10 == 0:  print("Error in step ", step," = ",e)
        if e < 0.001:
            break

    return P, Q

In [88]:
K = 5
import numpy as np
P = np.random.rand(train.shape[0],K)
Q = np.random.rand(K,train.shape[1])

P_n, Q_n = matrix_factorization(train, P, Q, K, alpha=0.0002, lambda_=0, steps=100)

R_hat = np.dot(P_n, Q_n)


  0%|          | 0/100 [00:00<?, ?it/s]

Error in step  0  =  430779.4843489156
Error in step  10  =  111479.10001539809
Error in step  20  =  94319.91901676983
Error in step  30  =  88479.4840334192
Error in step  40  =  85573.40487232569
Error in step  50  =  83839.66435068319
Error in step  60  =  82684.23286692737
Error in step  70  =  81852.30079644795
Error in step  80  =  81216.49360670471
Error in step  90  =  80706.0765196383


In [93]:
r = np.random.randint(0, len(test_data ), 3)
samples = test_data[r]
for i in range(samples.shape[0]):
  predicted = R_hat[samples[i][0],samples[i][1]]
  true = test[samples[i][0],samples[i][1]]
  print("Predicted label= ",predicted)
  print("True label= ",true)


Predicted label=  3.6489829539701155
True label=  3.0
Predicted label=  2.4594598973129487
True label=  2.0
Predicted label=  3.163521385913409
True label=  3.0


### Question 5

In [96]:
K = 5
import numpy as np
P = np.random.rand(train.shape[0],K)
Q = np.random.rand(K,train.shape[1])

P_n, Q_n = matrix_factorization(train, P, Q, K, alpha=0.0002, lambda_=0.01, steps=100)

R_hat2 = np.dot(P_n, Q_n)

  0%|          | 0/100 [00:00<?, ?it/s]

Error in step  0  =  428221.2472860305
Error in step  10  =  112796.40439199761
Error in step  20  =  94576.15288183832
Error in step  30  =  88439.34113599236
Error in step  40  =  85444.45744926982
Error in step  50  =  83684.3510163123
Error in step  60  =  82520.63566928817
Error in step  70  =  81683.10623364341
Error in step  80  =  81038.46471351478
Error in step  90  =  80513.45986663675


In [97]:
r = np.random.randint(0, len(test_data ), 3)
samples = test_data[r]
for i in range(samples.shape[0]):
  predicted = R_hat2[samples[i][0],samples[i][1]]
  true = test[samples[i][0],samples[i][1]]
  print("Predicted label= ",predicted)
  print("True label= ",true)

Predicted label=  4.403392056966718
True label=  5.0
Predicted label=  2.914162979849067
True label=  2.0
Predicted label=  3.408442872003556
True label=  3.0


### Question 7

In [98]:
def get_lambda(lambda_ , alpha, lambda_b, alpha_b, K ):
    lambdas_P = np.full((K,),lambda_[0])
    lambdas_Q = np.full((K,),lambda_[1])
    alphas_P =  np.full((K,),alpha[0])
    alphas_Q =  np.full((K,),alpha[1])

    lambdas_P[0] = 0
    lambdas_P[1] = lambda_b[0]
    lambdas_Q[0] = 0
    lambdas_Q[1] = lambda_b[1]
    alphas_P[0] = 0
    alphas_P[1] = alpha_b[0]
    alphas_Q[0] = 0
    alphas_Q[1] = alpha_b[1]
     
    return lambdas_P,lambdas_Q,alphas_P,alphas_Q


In [99]:
from tqdm.notebook import tqdm
def matrix_factorization2(R, P, Q, K, alpha_b=(0.0002, 0.0002), alpha =(0.0002, 0.0002),lambda_b=(0.01, 0.01), lambda_=(0.05, 0.05), steps=5000):
    lambdas_P,lambdas_Q,alphas_P,alphas_Q = get_lambda(lambda_ , alpha, lambda_b, alpha_b, K )
    P[:, 0] = np.ones((P.shape[0],))
    Q[:, 1] = np.ones((Q.shape[0],))
    E = R - np.dot(P, Q)

    for step in tqdm(range(steps)):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    for k in range(K):
                        P[i][k] +=  alphas_P[k] * (2 * E[i][j] * Q[k][j] - lambdas_P[k] * P[i][k])
                        Q[k][j] += alphas_Q[k] * (2 * E[i][j] * P[i][k] - lambdas_Q[k] * Q[k][j])


        e = 0
        R_hat = np.dot(P,Q)
        R_hat[R == 0] = 0
        E = R - R_hat
        e = np.sum(np.power(E,2))  + (lambda_[1]/2) * np.sum(np.power(Q,2)) + (lambda_[0]/2) * np.sum(np.power(P,2))
        if (step) % 10 == 0:  print("Error in step ", step," = ",e)
        if e < 0.001:
            break

    return P, Q

In [100]:
P = np.random.rand(train.shape[0],K)
Q = np.random.rand(K,train.shape[1])

P_n, Q_n = matrix_factorization2(train, P, Q, K, steps=100)

R_hat3 = np.dot(P_n, Q_n)

  0%|          | 0/100 [00:00<?, ?it/s]

Error in step  0  =  392815.5099359304
Error in step  10  =  112033.7910478636
Error in step  20  =  91913.50912486578
Error in step  30  =  85114.04761344203
Error in step  40  =  81827.42812674532
Error in step  50  =  79938.19110422494
Error in step  60  =  78725.90787387322
Error in step  70  =  77884.06604492232
Error in step  80  =  77262.53143092238
Error in step  90  =  76779.9508617338


In [105]:
r = np.random.randint(0, len(test_data ), 3)
samples = test_data[r]
for i in range(samples.shape[0]):
  predicted = R_hat3[samples[i][0],samples[i][1]]
  true = test[samples[i][0],samples[i][1]]
  print("Predicted label= ",predicted)
  print("True label= ",true)

Predicted label=  4.581264980874014
True label=  4.0
Predicted label=  3.859401755037764
True label=  5.0
Predicted label=  4.746401920814712
True label=  5.0


### Question 8

In [106]:
def give_parameter(alpha_b , alpha ,lambda_b ,lambda_, R ,K):
      if alpha_b is None:
          ps = [0.0002 for _ in range(7)]
      else:
          ps = alpha_b
      alpha_b = (ps[0] + ps[1] / np.log(R.shape[0])) + (ps[2] / np.sqrt(R.shape[0]) + ps[3] / R.shape[0] +ps[4] / np.log(R.shape[1])) + (ps[5] / np.sqrt(R.shape[1]) + ps[6] / R.shape[1])
      alpha_b = np.full((K,),alpha_b )

      if alpha is None:
          ps  = [0.0002 for _ in range(7)]
      else:
          ps = alpha
      alpha = (ps[0] + ps[1] / np.log(R.shape[0])) + (ps[2] / np.sqrt(R.shape[0]) + ps[3] / R.shape[0] +ps[4] / np.log(R.shape[1])) + (ps[5] / np.sqrt(R.shape[1]) + ps[6] / R.shape[1])
      alpha = np.full((K,),alpha)

      if lambda_b is None:
          ps = [0.0005 for _ in range(7)]
          
      else:
          ps  = lambda_b
      lambda_b = (ps[0] + ps[1] / np.log(R.shape[0])) + (ps[2] / np.sqrt(R.shape[0]) + ps[3] / R.shape[0] +ps[4] / np.log(R.shape[1])) + (ps[5] / np.sqrt(R.shape[1]) + ps[6] / R.shape[1])
      lambda_b = np.full((K,),lambda_b)

      if lambda_ is None:
          ps = [0.0005 for _ in range(7)]
      else:
          ps = lambda_
      lambda_ = (ps[0] + ps[1] / np.log(R.shape[0])) + (ps[2] / np.sqrt(R.shape[0]) + ps[3] / R.shape[0] +ps[4] / np.log(R.shape[1])) + (ps[5] / np.sqrt(R.shape[1]) + ps[6] / R.shape[1])
      lambda_ = np.full((K,),lambda_)

      return alpha_b , alpha ,lambda_b ,lambda_

In [107]:
from tqdm.notebook import tqdm
def matrix_factorization3(R, P, Q, K, alpha_b=None, alpha=None, lambda_b=None, lambda_=None, steps=5000):
    alpha_b , alpha ,lambda_b ,lambda_ = give_parameter(alpha_b , alpha ,lambda_b ,lambda_, R , K)
    lambdas_P,lambdas_Q,alphas_P,alphas_Q = get_lambda(lambda_ , alpha, lambda_b, alpha_b, K )
    P[:, 0] = np.ones((P.shape[0],))
    Q[:, 1] = np.ones((Q.shape[0],))
    E = R - np.dot(P, Q)

    for step in tqdm(range(steps)):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    for k in range(K):
                        P[i][k] +=  alphas_P[k] * (2 * E[i][j] * Q[k][j] - lambdas_P[k] * P[i][k])
                        Q[k][j] += alphas_Q[k] * (2 * E[i][j] * P[i][k] - lambdas_Q[k] * Q[k][j])


        e = 0
        R_hat = np.dot(P,Q)
        R_hat[R == 0] = 0
        E = R - R_hat
        e = np.sum(np.power(E,2))  + (lambda_[1]/2) * np.sum(np.power(Q,2)) + (lambda_[0]/2) * np.sum(np.power(P,2))
        if (step) % 10 == 0:  print("Error in step ", step," = ",e)
        if e < 0.001:
            break

    return P, Q

In [108]:
P = np.random.rand(train.shape[0],K)
Q = np.random.rand(K,train.shape[1])

P_n, Q_n = matrix_factorization3(train, P, Q, K, steps = 100)

R_hat4 = np.dot(P_n, Q_n)


  0%|          | 0/100 [00:00<?, ?it/s]

Error in step  0  =  370874.88346021925
Error in step  10  =  100093.51086230252
Error in step  20  =  86183.44827801043
Error in step  30  =  81547.70104425192
Error in step  40  =  79259.40868699056
Error in step  50  =  77887.23722678467
Error in step  60  =  76954.05384520529
Error in step  70  =  76256.20473021966
Error in step  80  =  75691.88720273589
Error in step  90  =  75204.20655836441


In [114]:
r = np.random.randint(0, len(test_data ), 3)
samples = test_data[r]
for i in range(samples.shape[0]):
  predicted = R_hat4[samples[i][0],samples[i][1]]
  true = test[samples[i][0],samples[i][1]]
  print("Predicted label= ",predicted)
  print("True label= ",true)

Predicted label=  3.0664966272680303
True label=  4.0
Predicted label=  2.34010065687722
True label=  2.0
Predicted label=  3.1722371269138696
True label=  3.0
